# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

In [7]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
df = pd.read_csv('../output_data/cities.csv')
df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Min Temp,Max Temp,Wind Speed
0,5882953,Aklavik,100,CA,2021-02-08 00:18:04,76,68.2191,-135.0107,-23.800,-22.000,76
1,3155487,Glomfjord,40,NO,2021-02-08 00:21:08,64,66.8167,13.9667,12.002,28.994,64
2,4032420,Neiafu,75,TO,2021-02-08 00:21:09,94,-18.6500,-173.9833,77.000,77.000,94
3,3874787,Punta Arenas,90,CL,2021-02-08 00:21:10,34,-53.1500,-70.9167,66.200,66.200,34
4,2088163,Popondetta,100,PG,2021-02-08 00:21:11,86,-8.7537,148.2534,80.078,80.078,86
...,...,...,...,...,...,...,...,...,...,...,...
541,1684245,Tagusao,97,PH,2021-02-08 00:33:17,78,9.1924,117.8141,81.518,81.518,78
542,3723593,Jérémie,27,HT,2021-02-08 00:33:19,71,18.6500,-74.1167,76.982,76.982,71
543,5975004,High Level,20,CA,2021-02-08 00:33:20,63,58.5169,-117.1360,-16.600,-16.600,63
544,2063036,Port Lincoln,58,AU,2021-02-08 00:33:21,71,-34.7333,135.8667,64.220,64.220,71


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure access to the GoogleMaps API.
gmaps.configure(api_key=g_key)

In [5]:
# Create a gmaps figure
fig = gmaps.figure()

# Create and add a heatmap layer
fig.add_layer(gmaps.heatmap_layer(
    df[["Lat", "Lng"]], 
    weights=df["Humidity"], 
    max_intensity=df["Humidity"].max(),
    dissipating=False, 
    point_radius=2
))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Heatmap
![heatmap image](../Images/heatmap.png)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# Narrow down the cities to fit weather conditions.
def fit_weather_criteria(data, temp_min, temp_max, wind_max):
    """
    Filter the data with by a temperature min (F), temperature max (F), and wind speed max (mph)
    
    Parameters
    ----------
        data — The original dataframe to filter from
        temp_min — temperature min (F)
        temp_max — temperature max (F)
        wind_max — wind speed max (mph)
    """
    # Drop any rows will null values
    filtered = data.dropna(inplace=False)
    # A min and max temperature
    temp_filter = (filtered['Max Temp'] >= temp_min) & (filtered['Max Temp'] <= temp_max)
    # Wind speed less max
    wind_filter = filtered['Wind Speed'] <= wind_max
    # send back the selected filterd
    return filtered.loc[temp_filter & wind_filter].reset_index(drop=True)

def get_some_filtered_data(data, min_amount=5):
    """
    Filter the data with decreasing strictness untill at least min amount remains
    
    Parameters
    ----------
        data — The original dataframe to filter from
    """
    temp_min = 70
    temp_max = 80
    wind_max = 10
    # test fit
    filtered = fit_weather_criteria(data, temp_min, temp_max, wind_max)
    # loop thru untill find at least min amount
    while (filtered.shape[0] < min_amount):
        temp_min -= 1
        temp_max += 1
        wind_max += 1
        filtered = fit_weather_criteria(data, temp_min, temp_max, wind_max)
    # inform user
    print(f'filtered from min temp of {temp_min}(°F), max temp of {temp_max}(°F), and max wind speed of {wind_max}(mph)')
    # return filtered
    return filtered

filtered_df = get_some_filtered_data(df)
filtered_df

filtered from min temp of 62(°F), max temp of 88(°F), and max wind speed of 18(mph)


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Min Temp,Max Temp,Wind Speed
0,2447513,Arlit,100,NE,2021-02-08 00:20:34,18,18.7369,7.3853,69.548,69.548,18
1,5525042,Levelland,20,US,2021-02-08 00:24:18,10,33.5873,-102.3780,69.800,69.800,10
2,2338660,Yaan,96,NG,2021-02-08 00:25:51,18,7.3833,8.5667,77.090,77.090,18
3,5546220,Saint George,1,US,2021-02-08 00:28:21,14,37.1041,-113.5841,62.600,62.600,14
4,2294877,Tamale,1,GH,2021-02-08 00:30:24,16,9.4008,-0.8393,83.534,83.534,16
5,3992405,Pitiquito,24,MX,2021-02-08 00:28:30,15,30.7000,-112.0833,73.994,73.994,15


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
google_maps_api_url = 'https://maps.googleapis.com/'

def google_maps_api_call(longitude, latitude, radius, key, types=None, as_json=True):
    """
    Call the google API formated for the Place Search.
    This returns a get request
    
    Parameters
    ----------
        key — Your application's API key. This key identifies your application. See Get a key for more information.
        location — The latitude/longitude around which to retrieve place information. This must be specified as latitude,longitude.
        radius — Defines the distance (in meters) within which to return place results. The maximum allowed radius is 50 000 meters.
        as_json — Returns the data as Json or XML
    """
    payload = {
        'location' : f'{longitude},{latitude}',
        'radius' : radius
    }
    # add types
    if type(types) is list and len(types) > 0:
        if len(types) == 1:
            payload['types'] = types
        else:
            payload['types'] = ','.join(types)
    # set the endpoint with the desired type
    endpoint = 'maps/api/place/nearbysearch/'
    if as_json: endpoint += 'json'
    else: endpoint += 'xml'
    # finally add the API key
    payload['key'] = key
    time.sleep(1)
    # make the call!
    return requests.get(google_maps_api_url + endpoint, params=payload)

In [8]:
hotel_df = pd.DataFrame(filtered_df)
hotel_df['Hotel Name'] = ['' for x in range(filtered_df.shape[0])]

# iterate thru rows
for index, row in filtered_df.iterrows():
    # print status
    status = f'{(index + 1)} of {filtered_df.shape[0]}'
    print(f"({status}) pinging server for city {row['City']}")
    # try to get request
    try:
        request = google_maps_api_call(
            longitude=row['Lat'],
            latitude=row['Lng'],
            radius=5000,
            key=g_key,
            types='lodging'
        )
        # if request fails, exit
        if request.status_code != 200:
            print(f"\tCould not find city {row['City']}, error {request.status_code}")
            continue
        # set the hotel name
        hotel_name = request.json()['results'][0]['name']
        hotel_df.loc[index,'Hotel Name'] = hotel_name
        # inform the user
        print(f"\tFound hotel {hotel_name} in city {row['City']}")
    except Exception as e:
        print(f"\tError - could not ping city {row['City']}, ERROR: {e}")

hotel_df

(1 of 6) pinging server for city Arlit
	Found hotel Arlit in city Arlit
(2 of 6) pinging server for city Levelland
	Found hotel Levelland in city Levelland
(3 of 6) pinging server for city Yaan
	Found hotel Iorjan in city Yaan
(4 of 6) pinging server for city Saint George
	Found hotel St. George in city Saint George
(5 of 6) pinging server for city Tamale
	Found hotel Tamale in city Tamale
(6 of 6) pinging server for city Pitiquito
	Found hotel Caborca in city Pitiquito


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Min Temp,Max Temp,Wind Speed,Hotel Name
0,2447513,Arlit,100,NE,2021-02-08 00:20:34,18,18.7369,7.3853,69.548,69.548,18,Arlit
1,5525042,Levelland,20,US,2021-02-08 00:24:18,10,33.5873,-102.3780,69.800,69.800,10,Levelland
2,2338660,Yaan,96,NG,2021-02-08 00:25:51,18,7.3833,8.5667,77.090,77.090,18,Iorjan
3,5546220,Saint George,1,US,2021-02-08 00:28:21,14,37.1041,-113.5841,62.600,62.600,14,St. George
4,2294877,Tamale,1,GH,2021-02-08 00:30:24,16,9.4008,-0.8393,83.534,83.534,16,Tamale
5,3992405,Pitiquito,24,MX,2021-02-08 00:28:30,15,30.7000,-112.0833,73.994,73.994,15,Caborca


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
![heatmap image](../Images/hotel_map.png)